In [2]:
#Book Recommendation System - Hybrid Model
# Dataset: Custom (20 books, 30+ ratings)

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

books_data = {
    "book_id": list(range(1, 21)),
    "title": [
        "The Alchemist", "Harry Potter", "Atomic Habits", "The Hobbit", "Deep Work",
        "Rich Dad Poor Dad", "1984", "To Kill a Mockingbird", "Becoming", "Educated",
        "The Power of Habit", "Thinking Fast and Slow", "The Silent Patient", "The Great Gatsby",
        "Sapiens", "The Subtle Art of Not Giving a F*ck", "Ikigai", "The Da Vinci Code",
        "Pride and Prejudice", "Crime and Punishment"
    ],
    "author": [
        "Paulo Coelho", "J.K. Rowling", "James Clear", "J.R.R. Tolkien", "Cal Newport",
        "Robert Kiyosaki", "George Orwell", "Harper Lee", "Michelle Obama", "Tara Westover",
        "Charles Duhigg", "Daniel Kahneman", "Alex Michaelides", "F. Scott Fitzgerald",
        "Yuval Noah Harari", "Mark Manson", "Héctor García", "Dan Brown",
        "Jane Austen", "Fyodor Dostoevsky"
    ],
    "genre": [
        "Fiction", "Fantasy", "Self-help", "Fantasy", "Productivity",
        "Finance", "Dystopian", "Classic", "Biography", "Memoir",
        "Self-help", "Psychology", "Thriller", "Classic",
        "History", "Self-help", "Philosophy", "Mystery",
        "Classic", "Philosophy"
    ],
    "description": [
        "A young shepherd's journey to find treasure in Egypt.",
        "A young wizard's magical education and battle against evil.",
        "Guide to habit formation for personal growth.",
        "A hobbit's adventure with dwarves and dragons.",
        "Strategies to focus in a distracted world.",
        "Lessons on wealth and investing from two fathers.",
        "A dystopian future under constant surveillance.",
        "A young girl's experience of racism and justice.",
        "The memoir of the former First Lady of the USA.",
        "A woman's journey from rural Idaho to academia.",
        "Explains the science behind why we do what we do.",
        "Explores human decision making and thinking systems.",
        "A woman’s shocking revelation about a murder.",
        "A mysterious millionaire and the American dream.",
        "A brief history of humankind’s evolution.",
        "Counterintuitive advice to live a better life.",
        "Japanese secret to a long and happy life.",
        "A symbologist unravels a murder mystery in Paris.",
        "A classic tale of love and societal expectations.",
        "A man's moral dilemmas and redemption journey."
    ]
}

ratings_data = {
    "user_id": [
        1,1,1,2,2,2,3,3,3,4,4,
        5,5,5,6,6,6,7,7,8,8,
        9,9,10,10,11,11,12,12,13,13
    ],
    "book_id": [
        1, 2, 3, 2, 4, 5, 1, 6, 7, 2, 8,
        9, 10, 1, 11, 12, 13, 1, 14, 3, 15,
        16, 17, 18, 19, 1, 20, 6, 14, 5, 2
    ],
    "rating": [
        5, 4, 5, 5, 4, 3, 5, 3, 4, 4, 3,
        4, 5, 5, 5, 4, 4, 4, 4, 5, 5,
        3, 4, 4, 5, 4, 5, 4, 3, 4, 4
    ]
}

books_df = pd.DataFrame(books_data)
ratings_df = pd.DataFrame(ratings_data)

# 1.. Content-Based Filtering
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books_df['description'])
content_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
book_indices = pd.Series(books_df.index, index=books_df['title'])

# 2. Collaborative Filtering
user_book_matrix = ratings_df.pivot_table(index='user_id', columns='book_id', values='rating')
user_book_matrix = user_book_matrix.fillna(0)
collab_sim = cosine_similarity(user_book_matrix.T)
collab_book_idx = pd.Series(user_book_matrix.columns)

# 4. Recommendation Functions
def recommend_content(title, top_n=5):
    idx = book_indices[title]
    sim_scores = list(enumerate(content_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    book_ids = [i[0] for i in sim_scores]
    return books_df.iloc[book_ids][['title', 'author', 'genre']]

def recommend_collab(book_id, top_n=5):
    idx = collab_book_idx[collab_book_idx == book_id].index[0]
    sim_scores = list(enumerate(collab_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    book_ids = [collab_book_idx[i] for i, _ in sim_scores]
    return books_df[books_df['book_id'].isin(book_ids)][['title', 'author', 'genre']]

def recommend_hybrid(title, top_n=5, alpha=0.5):
    idx_content = book_indices[title]
    book_id = books_df.loc[idx_content, 'book_id']
    idx_collab = collab_book_idx[collab_book_idx == book_id].index[0]
    hybrid_score = alpha * content_sim[idx_content] + (1 - alpha) * collab_sim[idx_collab]
    sim_scores = list(enumerate(hybrid_score))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    book_ids = [i[0] for i in sim_scores]
    return books_df.iloc[book_ids][['title', 'author', 'genre']]

# 5. Test the Recommenders
print("\nContent-based Recommendations for 'Atomic Habits':")
print(recommend_content("Atomic Habits"))

print("\nCollaborative Recommendations for Book ID 3 (Atomic Habits):")
print(recommend_collab(3))

print("\nHybrid Recommendations for 'Atomic Habits':")
print(recommend_hybrid("Atomic Habits", alpha=0.5))



Content-based Recommendations for 'Atomic Habits':
               title           author         genre
0      The Alchemist     Paulo Coelho       Fiction
1       Harry Potter     J.K. Rowling       Fantasy
3         The Hobbit   J.R.R. Tolkien       Fantasy
4          Deep Work      Cal Newport  Productivity
5  Rich Dad Poor Dad  Robert Kiyosaki       Finance

Collaborative Recommendations for Book ID 3 (Atomic Habits):
            title             author         genre
0   The Alchemist       Paulo Coelho       Fiction
1    Harry Potter       J.K. Rowling       Fantasy
3      The Hobbit     J.R.R. Tolkien       Fantasy
4       Deep Work        Cal Newport  Productivity
14        Sapiens  Yuval Noah Harari       History

Hybrid Recommendations for 'Atomic Habits':
            title             author         genre
14        Sapiens  Yuval Noah Harari       History
0   The Alchemist       Paulo Coelho       Fiction
1    Harry Potter       J.K. Rowling       Fantasy
3      The Hobbit  